In [11]:
%reload_ext autoreload
%autoreload 2

from loader.build_loader import build_loader_SNP
from model.build_model import build_model
from inference.build_inference import build_inference_SNP
from utils import ensure_dir
import logging, coloredlogs
import argparse
import yaml
import os
import torch
import requests

os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"
response = requests.get('http://www.google.com')
print(response.status_code)

# setting parser
parser = argparse.ArgumentParser()
parser.add_argument('--cfg_path', type=str, default='configs/')
parser.add_argument('--checkpoint', type=str, default=None)
parser.add_argument('--eval_only', action='store_true', default=False)
opt = parser.parse_args(args=[])  # Avoid parsing command-line arguments

opt.cfg_path = 'configs/vg_msdn/vg_msdn_seq2seq_v24.yaml'
opt.checkpoint = './saved/vg_msdn_F_seq2seq_v24/checkpoint_50_0.16316922369277578.pth'
opt.eval_only = True

# setting config file
with open(opt.cfg_path, 'r') as f:
    cfg = yaml.safe_load(f)

# handle dir for saving
ensure_dir(cfg['OUTPUT']['LOG_DIR'])
ensure_dir(cfg['TEST']['SAVE_DIR'])

# setting logger
handlers = [logging.FileHandler(os.path.join(cfg['OUTPUT']['OUTPUT_DIR'],
                                             'output_eval.log'), mode='w'), logging.StreamHandler()]
logging.basicConfig(handlers=handlers, level=logging.INFO)
logger = logging.getLogger('root')
coloredlogs.install(logger=logger, fmt='%(asctime)s [%(name)s] %(levelname)s %(message)s')
logger.info('Setup output directory - {}.'.format(cfg['OUTPUT']['LOG_DIR']))

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

single_data, _ = build_loader_SNP(cfg, SCENE_NM='cat', PROMPT='a cat sits on a table', CANVAS_SIZE=(224, 224))
print("load data done.")
rel2bbox_model = build_model(cfg)  # unchanged, accord with inference checkpoint
print("build model done.")
assert opt.checkpoint is not None, 'Please provide model ckpt for testing'
checkpoint = torch.load(opt.checkpoint)
rel2bbox_model.load_state_dict(checkpoint['state_dict'])
print("load checkpoint done.")
infer_pipeline = build_inference_SNP(cfg, device)
print("build inference pipeline done.")
infer_pipeline.run_single(cfg=cfg, model=rel2bbox_model, dataset=single_data.dataset)


ModuleNotFoundError: No module named 'model.build_model'